#  Training a SSD MobileNet V2 for Pedestrian Detection with TensorFlow Object Detection API
This notebook shows how to train a SSD MobileNet V2 object detector for pedestrian detection with [TensorFlow Object Detection API](https://github.com/tensorflow/models/tree/master/research/object_detection). The TensorFlow Object Detection API is a framework for training object detection models that offers a lot of flexibility.
0. [Set up env variables](#head-0)
1. [Download and Prepare Dataset](#head-1)
2. [Create tfrecords](#head-2)
3. [Download Pre-Trained Models](#head-3)
4. [Model Training](#head-4)
5. [Export the Model for Inference](#head-5) <br>
5.1. [General Case](#head-5-1) <br>
5.2. [SSD Model That Supposed to be Export to tflite](#head-5-2) <br>

## 0. Set up env variables <a class=anchor id=head-0></a>


In [ ]:
%set_env DATASET_DIR=/experiment_dir/dataset
%set_env PRETRAINED_MODELS_DIR =/experiment_dir/pretrained_models

## 1. Download and Prepare Dataset <a class=anchor id=head-1></a>
We will be using [Oxford Town Center Dataset](https://www.robots.ox.ac.uk/ActiveVision/Research/Projects/2009bbenfold_headpose/project.html) for training our pedestrian detection. First we will download the video file and annotatoin csv file with `download_towncenter_video_and_labels.sh` script and then extract frames from video with [ffmpeg](https://ffmpeg.org/) tool and xml annotation files with `create_tfrecord.py` script.

In [ ]:
# Download Oxford Town Center Dataset video file and csv annotation file
!bash ./download_towncenter_video_and_labels.sh

In [ ]:
# Extract images from video and save in images directory with `ffmpeg`
!mkdir $DATASET_DIR/images
!ffmpeg -i $DATASET_DIR/TownCentreXVID.avi -q:v 1 -start_number 0 -frames:v 4501 $DATASET_DIR/images/%d.jpg

In [ ]:
# Create xml annotation files and save in xmls directory
!python create_xmls.py --annotation_path $DATASET_DIR/TownCentre-groundtruth.top

## 2. Create `tfrecords` <a class=anchor id=head-2></a>
TensorFlow Object detection API only accepts `tfrecord` file format for loading and reading datasets. So we wrote a python script to create `tfrecord` files from our images and annotations. it will create two `tfrecord` files for training and evaluation set in the `output_dir`. you can specify how to split train/eval sets by setting `validation_split`

In [ ]:
!python create_tfrecord.py \
--data_dir $DATASET_DIR \
--output_dir $DATASET_DIR \
--label_map_path ./label_map.pbtxt \
--validation_split 0.25

## 3. Download Pre-Trained Models <a class=anchor id=head-3></a>
TensorFlow provides large number of pre-trained models which is trained on various datasets. So you don't need to start training from scratch and you can use these models as your starting point. You just need to download one of the models from [TensorFlow Object Detection Model Zoo](https://github.com/tensorflow/models/blob/master/research/object_detection/g3doc/detection_model_zoo.md), unzip the file and specify its path in the training config file.

In [ ]:
!mkdir $PRETRAINED_MODELS_DIR

In [ ]:
# For ssd_mobilenet_v2 and ssd_mobilenet_v2_quantized
!wget http://download.tensorflow.org/models/object_detection/ssd_mobilenet_v2_coco_2018_03_29.tar.gz -O $PRETRAINED_MODELS_DIR/ssd_mobilenet_v2_coco_2018_03_29.tar.gz
# For ssdlite_mobilenet_v2
!wget http://download.tensorflow.org/models/object_detection/ssdlite_mobilenet_v2_coco_2018_05_09.tar.gz -O $PRETRAINED_MODELS_DIR/ssdlite_mobilenet_v2_coco_2018_05_09.tar.gz
# For faster_rcnn_resnet50
!wget http://download.tensorflow.org/models/object_detection/faster_rcnn_resnet50_coco_2018_01_28.tar.gz -O $PRETRAINED_MODELS_DIR/faster_rcnn_resnet50_coco_2018_01_28.tar.gz

In [ ]:
!tar -xzvf $PRETRAINED_MODELS_DIR/ssd_mobilenet_v2_coco_2018_03_29.tar.gz -C $PRETRAINED_MODELS_DIR

## 4. Model Training <a class=anchor id=head-4></a>
By specifying the training config file, the model directory which stores the training checkpoints and logs and number of training steps you can start training the model.
note that you can track the training procedure with tensorboard by running `tensorboar --logdir [MODEL_DIR]` in training docker container.

In [ ]:
!mkdir /experiment_dir/ped_ssd_mobilenet_v2
%set_env PIPELINE_CONFIG_PATH=/repo/training/tf_object_detection_api/pedestrian_detection/configs/ssd_mobilenet_v2_pedestrian.config
%set_env MODEL_DIR=/experiment_dir/ped_ssd_mobilenet_v2
%set_env NUM_TRAIN_STEPS=50000
%set_env SAMPLE_1_OF_N_EVAL_EXAMPLES=1

In [ ]:
!python /models/research/object_detection/model_main.py --pipeline_config_path=${PIPELINE_CONFIG_PATH} --model_dir=${MODEL_DIR} --num_train_steps=${NUM_TRAIN_STEPS} --sample_1_of_n_eval_examples=$SAMPLE_1_OF_N_EVAL_EXAMPLES --alsologtostderr

## 5. Export the Model for Inference <a class=anchor id=head-5></a>
After the model has been trained you should export the trained model to a frozen graph `.pb` file so you can use it for inference.
Note that if your model is SSD and it is supposed to be converted to `tflite`, you should use `export_tflite_ssd_graph.py` script to export the model. (section 5.2)

### 5.1 General Case <a class=anchor id=head-5-1></a>

In [ ]:
# Set the step number which has best performance
STEP=33387

In [ ]:
!mkdir -p $MODEL_DIR/frozen_graph
%set_env INPUT_TYPE=image_tensor
%set_env TRAINED_CKPT_PREFIX=/experiment_dir/ped_ssd_mobilenet_v2/model.ckpt-{STEP}
%set_env EXPORT_DIR=/experiment_dir/ped_ssd_mobilenet_v2/frozen_graph

In [ ]:
!python /models/research/object_detection/export_inference_graph.py \
    --input_type=${INPUT_TYPE} \
    --pipeline_config_path=${PIPELINE_CONFIG_PATH} \
    --trained_checkpoint_prefix=${TRAINED_CKPT_PREFIX} \
    --output_directory=${EXPORT_DIR}

In [ ]:
!ls $MODEL_DIR

### 5.2 SSD Model That Supposed to be Export to `tflite` <a class=anchor id=head-5-2></a>

In [ ]:
# Set the step number which has best performance
STEP=33387

In [ ]:
!mkdir -p $MODEL_DIR/frozen_graph_tflite
%set_env OUTPUT_DIR=/experiment_dir/ped_ssd_mobilenet_v2/frozen_graph_tflite
%set_env TRAINED_CKPT_PREFIX=/experiment_dir/ped_ssd_mobilenet_v2/model.ckpt-{STEP}

In [ ]:
!python /models/research/object_detection/export_tflite_ssd_graph.py \
--pipeline_config_path=$PIPELINE_CONFIG_PATH \
--trained_checkpoint_prefix=$TRAINED_CKPT_PREFIX \
--output_directory=$OUTPUT_DIR \
--add_postprocessing_op=true \
--max_detections=50

In [ ]:
!ls $MODEL_DIR/frozen_graph_tflite

In case your model is trained with quantizatoin-aware training strategy you can convert the `.pb` frozen graph to a quantized `.tflite` model with our [docker container](https://github.com/neuralet/neuralet/blob/master/training/tf_object_detection_api/tools-toco.Dockerfile)